In [1]:
import caffe
import cv2
import numpy as np
import os

def preprocess_frame(frame, input_size):
    # Resize the frame to the input size required by the model
    frame_resized = cv2.resize(frame, (input_size[1], input_size[0]))
    # Convert the frame to a blob suitable for Caffe
    blob = caffe.io.blobproto_to_array(frame_resized)
    # Transpose the blob dimensions to match Caffe's format (NCHW)
    blob = np.transpose(blob, (0, 3, 1, 2))
    return blob


ModuleNotFoundError: No module named 'caffe'

In [ ]:

def run_inference(caffe_net, frame_blob):
    # Set the input to the network
    caffe_net.blobs['data'].data[...] = frame_blob
    # Perform forward pass
    output = caffe_net.forward()
    # Extract the predictions
    predictions = output['prob']
    return predictions

In [ ]:

def process_frames(frames_dir, model_def, model_weights, input_size, output_dir):
    # Initialize Caffe
    caffe.set_mode_cpu()
    caffe_net = caffe.Net(model_def, model_weights, caffe.TEST)
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Process each frame in the frames directory
    for frame_file in os.listdir(frames_dir):
        frame_path = os.path.join(frames_dir, frame_file)
        if frame_path.lower().endswith(('.jpg', '.png', '.jpeg')):
            # Read the frame
            frame = cv2.imread(frame_path)
            # Preprocess the frame
            frame_blob = preprocess_frame(frame, input_size)
            # Run inference
            predictions = run_inference(caffe_net, frame_blob)
            # Save or process predictions (for demonstration, saving as text file)
            output_file = os.path.join(output_dir, f"{os.path.splitext(frame_file)[0]}_predictions.txt")
            with open(output_file, 'w') as f:
                f.write(str(predictions))
            print(f"Processed: {frame_file}")

    print("All frames processed.")


In [ ]:
frames_dir = 'extracted_frames'  # Replace with the directory of extracted frames
model_def = 'shanghaia.caffemodel'    # Replace with the path to the Caffe model definition
model_weights = 'path/to/model.caffemodel'  # Replace with the path to the Caffe model weights
input_size = (224, 224)  # Replace with the input size required by your model (height, width)
output_dir = 'new_dir'  # Replace with the directory to save predictions

process_frames(frames_dir, model_def, model_weights, input_size, output_dir)